In [82]:
#importing libraries

import cv2
import os
import numpy as np
import pandas as pd
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"
import import_ipynb
from imagePreProcessor import *
from TesseractModify import *
from datetime import datetime
import math
import dateparser
import datefinder
from IPython.display import clear_output
import difflib

In [83]:
imagesFolderPath = '/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Data/Images'

In [84]:
def preProcessDataFrame(df, image):
    
    #drop null values
    df.dropna(inplace=True)
    
    
    #rows having text as a stopword removed
    indexesToDrop = []
    stopwords = ["", " ", "  ", "/", ":", "-", ".", ",", "\n", "\t", "\\", "(", ")", "[", "]", "{", "}", "*", "&", "%", "$", "#", "|"]
    for index in df.index:
        if df["text"][index] in stopwords:
            indexesToDrop.append(index)   
    df.drop(indexesToDrop, inplace=True)
    
    #resetting the index
    df.reset_index(inplace=True)
    df.drop(columns=["index"], inplace=True)

    for index in df.index:
        df["text"][index] = df["text"][index].lower()
    
    #creating new columns for centroid information
    df["x"] = df["left"] + df["width"] 
    df["y"] = df["top"] + df["height"]
    
    #new columns for page width and height for further normalization
    df["PageHeight"] = image.shape[0]
    df["PageWidth"] = image.shape[1]
    
    #adding columns for zonal info (4 Zone)
    df["isTop"] = 0
    df["isBottom"] = 0
    df["isRight"] = 0
    df["isLeft"] = 0

In [85]:
def verifyDate(text):
    
    dates = list(datefinder.find_dates(text, strict=True))
    
    if len(dates)==0:
        return False
    else:
        for date in dates:
            year = date.year
            presentYear = date.today().year
            if abs(year-presentYear)<=2:
                print(dates)
                return True
            else:
                return False

In [86]:
def findDate(dateDF):
    dates = []
    
    for index in dateDF.index:
        text = dateDF["text"][index]
        if verifyDate(text):
            dates.append(text)
    return dates

In [87]:
def extractDateDataFrame(df):
    dateDF = df.copy()
    for index in dateDF.index:
        text = dateDF["text"][index]
        dateDF["text"][index] = text.lower()
    dates = findDate(dateDF)
    return dateDF, dates
    
        

In [88]:
def returnDist(x1, y1, x2, y2):
    p = [x1, y1]
    q = [x2, y2]
    return math.dist(p, q)

In [22]:
def compare(word, neighbours):
    if word in neighbours:
        return word
    else:
        matches = difflib.get_close_matches(word, neighbours, n=1, cutoff=0.8)
        if len(matches)==0:
            return None
        else:
            return matches[0]
            

In [222]:
def allocateNeighbours(df, neighbourFields):
    for index in df.index:
        x1 = df["x"][index]
        y1 = df["y"][index]
        block = df["block_num"][index]
        neighbours = {}
        for index2 in df.index:
            x2 = df["x"][index2]
            y2 = df["y"][index2]
            block2 = df["block_num"][index2]
            dist = returnDist(x1, y1, x2, y2)
            if x2<=x1 and y2<=y1:
                if df["text"][index2].lower() in neighbours:
                    neighbours[df["text"][index2].lower()]=min(dist, neighbours[df["text"][index2].lower()])
                else:
                    neighbours[df["text"][index2].lower()]=dist
            if (x2<=x1 and y2<=y1) and abs(block2-block)<=1:
                if df["text"][index2].lower() in neighbours:
                    neighbours[df["text"][index2].lower()]=min(dist, neighbours[df["text"][index2].lower()])
                else:
                    neighbours[df["text"][index2].lower()]=dist
        for n in neighbourFields:
            if compare(n, neighbours.keys())==None:
                pass
            else:
                df[n][index]=neighbours[compare(n, neighbours.keys())]

In [223]:
def addNeighbours(df):
    neighbourFields = ["date", "dated", "invoice", "inv", "delivery", "order", "due", "payment", "tax", "bill", "receipt", "issue", "invoice date", "invoice dt", "due date", "payment due date", "ack date", "ack dt", "inv date", "inv dt"]
    for col in neighbourFields:
        df[col]=0
    allocateNeighbours(df, neighbourFields)
    

In [224]:
def findDateDF(dates, dateDF):
    indexes = []
    for date in dates:
        i = dateDF[dateDF["text"]==date].index
        for index in i:
            indexes.append(index)
    return indexes

In [225]:
def dropIndexes(indexes, df):
    indexesToDrop=[]
    for index in df.index:
        if index not in indexes:
            indexesToDrop.append(index)
    df.drop(indexesToDrop, inplace=True)

In [226]:
def create(imagePath):
    image = cv2.imread(imagePath)
    preProcessedImage = preProcessImage(imagePath)
    data = pytesseract.image_to_data(preProcessedImage, output_type=Output.DATAFRAME)
    data.dropna(inplace=True)
    data.reset_index(inplace=True)
    data.drop(columns=["index"], inplace=True)
    processData(data)
    df = pd.DataFrame(data)
    preProcessDataFrame(df, preProcessedImage)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    return resultDF
    
    

In [227]:
def batchCreate(folderPath):
    os.chdir(folderPath)
    images = os.listdir()
    dfList = []
    for imagePath in images:
        #print(imagePath)
        df = create(imagePath)
        df["imageName"] = imagePath
        df["output"] = 0
        dfList.append(df)
    return dfList

In [228]:
# os.chdir(imagesFolderPath)

In [229]:
# imagePath = "1.jpeg"

In [230]:
# df = create(imagePath)
# df

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_12350/1412637721.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_12350/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_12350/22425511.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

[datetime.datetime(2022, 7, 21, 0, 0)]
[datetime.datetime(2022, 7, 21, 0, 0)]


,block_num,line_num,left,top,width,height,conf,text,nextDist,x,...,receipt,issue,invoice date,invoice dt,due date,payment due date,ack date,ack dt,inv date,inv dt
6,2,2,1713,256,203,28,96.087402,21-07-2022,-1749,1916,...,0,0,688.104643,688.104643,0,0,0,0,0,0
151,2,0,1713,256,203,28,96.087402,21-07-2022,-1749,1916,...,0,0,688.104643,688.104643,0,0,0,0,0,0


In [231]:
# dfList = batchCreate(imagesFolderPath)

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_12350/1412637721.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_12350/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_12350/22425511.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]


<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_12350/1412637721.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_12350/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()


[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]


/tmp/ipykernel_12350/22425511.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[compare(n, neighbours.keys())]
<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_12350/1412637721.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_12350/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

...

[datetime.datetime(2022, 12, 24, 0, 0)]
[datetime.datetime(2022, 12, 27, 0, 0)]
[datetime.datetime(2022, 12, 24, 0, 0)]
[datetime.datetime(2022, 12, 27, 0, 0)]


<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_12350/1412637721.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_12350/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_12350/22425511.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

[datetime.datetime(2023, 2, 14, 0, 0)]
[datetime.datetime(2023, 2, 14, 0, 0)]


<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_12350/1412637721.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_12350/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_12350/22425511.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

[datetime.datetime(2022, 9, 28, 0, 0)]
[datetime.datetime(2022, 9, 28, 0, 0)]


In [232]:
# len(dfList)

110

In [233]:
# resultDF = pd.concat(dfList)

In [234]:
# resultDF.describe()

,block_num,line_num,left,top,width,height,conf,nextDist,x,y,...,issue,invoice date,invoice dt,due date,payment due date,ack date,ack dt,inv date,inv dt,output
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,...,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0,299.0,299.0
mean,9.648829,1.632107,1235.505017,1136.431438,190.010033,32.411371,82.248156,-684.695652,1425.515050,1168.842809,...,29.739241,453.565473,576.111902,48.246929,3.083909,19.982051,68.894987,0.0,0.0,0.0
std,8.896521,3.393406,752.495037,738.282392,75.580371,7.629948,20.776512,954.989468,751.611335,738.461133,...,176.333260,707.224462,687.483932,234.537914,37.645885,178.352872,379.348725,0.0,0.0,0.0
min,1.000000,0.000000,75.000000,67.000000,98.000000,16.000000,0.000000,-3189.000000,210.000000,97.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
25%,4.000000,0.000000,556.000000,598.500000,154.000000,28.000000,74.454445,-1627.000000,783.500000,632.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
50%,6.000000,1.000000,1367.000000,915.000000,178.000000,32.000000,90.819194,-401.000000,1595.000000,937.000000,...,0.000000,0.000000,434.442171,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
75%,12.000000,2.000000,1908.000000,1515.500000,203.000000,37.000000,96.216804,118.000000,2113.000000,1554.500000,...,0.000000,596.505151,762.402125,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
max,46.000000,26.000000,3130.000000,3395.000000,821.000000,71.000000,96.945030,2035.000000,3278.000000,3424.000000,...,1361.009184,3226.750998,3226.750998,2075.708313,466.000000,2156.320013,2875.171125,0.0,0.0,0.0


In [235]:
# len(resultDF["imageName"].unique())

72

In [236]:
# resultDF.reset_index(inplace=True)

In [237]:
# resultDF.drop(columns=["index", "nextDist", "block_num", "line_num", "inv date", "inv dt"], inplace=True)

In [238]:
# resultDF.shape

(299, 34)

In [239]:
# resultDF.head()

,left,top,width,height,conf,text,x,y,PageHeight,PageWidth,...,receipt,issue,invoice date,invoice dt,due date,payment due date,ack date,ack dt,imageName,output
0,552,1142,269,34,46.287964,dot::05/12/2022,821,1176,3500,2525,...,229.538668,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,20.jpeg,0
1,552,1142,269,34,46.287964,dot::05/12/2022,821,1176,3500,2525,...,229.538668,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,20.jpeg,0
2,2005,536,179,28,96.255013,05/12/2022,2184,564,3300,2550,...,0.000000,0.0,0.0,627.072564,0.0,0.0,0.0,0.0,23.jpeg,0
3,2005,681,184,29,96.816185,05/12/2022,2189,710,3300,2550,...,0.000000,0.0,0.0,662.499057,0.0,0.0,0.0,0.0,23.jpeg,0
4,1820,1355,184,29,94.628899,05/12/2022,2004,1384,3300,2550,...,0.000000,0.0,0.0,987.451265,0.0,0.0,0.0,0.0,23.jpeg,0


In [240]:
# resultDF.describe()

,left,top,width,height,conf,x,y,PageHeight,PageWidth,isTop,...,bill,receipt,issue,invoice date,invoice dt,due date,payment due date,ack date,ack dt,output
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0,...,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0
mean,1235.505017,1136.431438,190.010033,32.411371,82.248156,1425.515050,1168.842809,3459.036789,2572.792642,0.0,...,282.010346,18.211885,29.739241,453.565473,576.111902,48.246929,3.083909,19.982051,68.894987,0.0
std,752.495037,738.282392,75.580371,7.629948,20.776512,751.611335,738.461133,464.527813,322.096359,0.0,...,638.679624,113.209609,176.333260,707.224462,687.483932,234.537914,37.645885,178.352872,379.348725,0.0
min,75.000000,67.000000,98.000000,16.000000,0.000000,210.000000,97.000000,1566.000000,1134.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,556.000000,598.500000,154.000000,28.000000,74.454445,783.500000,632.000000,3300.000000,2479.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,1367.000000,915.000000,178.000000,32.000000,90.819194,1595.000000,937.000000,3484.000000,2481.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,434.442171,0.000000,0.000000,0.000000,0.000000,0.0
75%,1908.000000,1515.500000,203.000000,37.000000,96.216804,2113.000000,1554.500000,3508.000000,2550.000000,0.0,...,0.000000,0.000000,0.000000,596.505151,762.402125,0.000000,0.000000,0.000000,0.000000,0.0
max,3130.000000,3395.000000,821.000000,71.000000,96.945030,3278.000000,3424.000000,5313.000000,3542.000000,0.0,...,2998.019680,933.944859,1361.009184,3226.750998,3226.750998,2075.708313,466.000000,2156.320013,2875.171125,0.0


In [241]:
# resultDF["contextPresent"]=0

In [242]:
# contexts = ["invoice date", "invoice", "invoice dt"]

In [243]:
# for index in resultDF.index:
#     cont = []
#     for col in contexts:
#         cont.append(resultDF[col][index])
#     if max(cont)>0:
#         resultDF["contextPresent"][index]=1

/tmp/ipykernel_12350/1552890533.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultDF["contextPresent"][index]=1


In [244]:
# resultDF.describe()

,left,top,width,height,conf,x,y,PageHeight,PageWidth,isTop,...,receipt,issue,invoice date,invoice dt,due date,payment due date,ack date,ack dt,output,contextPresent
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0,...,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0,299.000000
mean,1235.505017,1136.431438,190.010033,32.411371,82.248156,1425.515050,1168.842809,3459.036789,2572.792642,0.0,...,18.211885,29.739241,453.565473,576.111902,48.246929,3.083909,19.982051,68.894987,0.0,0.622074
std,752.495037,738.282392,75.580371,7.629948,20.776512,751.611335,738.461133,464.527813,322.096359,0.0,...,113.209609,176.333260,707.224462,687.483932,234.537914,37.645885,178.352872,379.348725,0.0,0.485682
min,75.000000,67.000000,98.000000,16.000000,0.000000,210.000000,97.000000,1566.000000,1134.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,556.000000,598.500000,154.000000,28.000000,74.454445,783.500000,632.000000,3300.000000,2479.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,1367.000000,915.000000,178.000000,32.000000,90.819194,1595.000000,937.000000,3484.000000,2481.000000,0.0,...,0.000000,0.000000,0.000000,434.442171,0.000000,0.000000,0.000000,0.000000,0.0,1.000000
75%,1908.000000,1515.500000,203.000000,37.000000,96.216804,2113.000000,1554.500000,3508.000000,2550.000000,0.0,...,0.000000,0.000000,596.505151,762.402125,0.000000,0.000000,0.000000,0.000000,0.0,1.000000
max,3130.000000,3395.000000,821.000000,71.000000,96.945030,3278.000000,3424.000000,5313.000000,3542.000000,0.0,...,933.944859,1361.009184,3226.750998,3226.750998,2075.708313,466.000000,2156.320013,2875.171125,0.0,1.000000


In [245]:
# resultDF["contextDist"] = 0

In [246]:
# for index in resultDF.index:
#     if resultDF["contextPresent"][index]==1:
#         cont = []
#         for col in contexts:
#             if resultDF[col][index]>0:
#                 cont.append(resultDF[col][index])
#         dist = min(cont)
#         resultDF["contextDist"][index] = dist


/tmp/ipykernel_12350/2068031429.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultDF["contextDist"][index] = dist


In [247]:
# resultDF.drop(columns=["output"], inplace=True)

In [248]:
# resultDF["output"] = 0

In [249]:
# resultDF

,left,top,width,height,conf,text,x,y,PageHeight,PageWidth,...,invoice date,invoice dt,due date,payment due date,ack date,ack dt,imageName,contextPresent,contextDist,output
0,552,1142,269,34,46.287964,dot::05/12/2022,821,1176,3500,2525,...,0.000000,0.000000,0.0,0.0,0.0,0.0,20.jpeg,0,0.000000,0
1,552,1142,269,34,46.287964,dot::05/12/2022,821,1176,3500,2525,...,0.000000,0.000000,0.0,0.0,0.0,0.0,20.jpeg,0,0.000000,0
2,2005,536,179,28,96.255013,05/12/2022,2184,564,3300,2550,...,0.000000,627.072564,0.0,0.0,0.0,0.0,23.jpeg,1,627.072564,0
3,2005,681,184,29,96.816185,05/12/2022,2189,710,3300,2550,...,0.000000,662.499057,0.0,0.0,0.0,0.0,23.jpeg,1,662.499057,0
4,1820,1355,184,29,94.628899,05/12/2022,2004,1384,3300,2550,...,0.000000,987.451265,0.0,0.0,0.0,0.0,23.jpeg,1,987.451265,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,2006,1630,171,26,82.106323,27-dec-22,2177,1656,3508,2479,...,1745.850223,1745.850223,0.0,0.0,0.0,0.0,86.jpeg,1,1107.792851,0
295,1704,225,181,31,90.515984,14-feb-23,1885,256,3300,2550,...,582.092776,582.092776,0.0,0.0,0.0,0.0,46.jpeg,1,548.820554,0
296,1704,225,181,31,90.515984,14-feb-23,1885,256,3300,2550,...,582.092776,582.092776,0.0,0.0,0.0,0.0,46.jpeg,1,548.820554,0
297,1696,208,194,42,96.214699,28-sep-22,1890,250,3508,2479,...,606.690201,606.690201,0.0,0.0,0.0,0.0,62.jpeg,1,606.690201,0


In [259]:
# for index in resultDF.index:
#     name = resultDF["imageName"][index].split(".")[0]
#     resultDF["imageName"][index]=int(name)

In [251]:
# resultDF.sort_values("imageName", inplace=True)
# resultDF

,left,top,width,height,conf,text,x,y,PageHeight,PageWidth,...,invoice date,invoice dt,due date,payment due date,ack date,ack dt,imageName,contextPresent,contextDist,output
72,1713,256,203,28,96.087402,21-07-2022,1916,284,3504,2479,...,688.104643,688.104643,0.0,0.0,0.000000,0.000000,1,1,688.104643,0
73,1713,256,203,28,96.087402,21-07-2022,1916,284,3504,2479,...,688.104643,688.104643,0.0,0.0,0.000000,0.000000,1,1,688.104643,0
206,1648,1936,214,31,79.921706,1-oct-22. to 31,1862,1967,3504,2479,...,0.000000,1077.894707,0.0,0.0,2156.320013,2156.320013,2,1,1077.894707,0
204,1648,1936,136,31,48.239994,1-oct-22.,1784,1967,3504,2479,...,0.000000,1049.665185,0.0,0.0,2099.243673,2099.243673,2,1,1049.665185,0
203,376,486,163,35,95.528168,22-sep-22,539,521,3504,2479,...,0.000000,0.000000,0.0,0.0,273.029302,273.029302,2,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2133,1209,185,30,95.900703,01-feb-2023,2318,1239,3300,2550,...,0.000000,1553.208936,0.0,0.0,0.000000,0.000000,97,1,1553.208936,0
270,2098,536,159,28,96.263855,21-08-2022,2257,564,3508,2479,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,98,0,0.000000,0
88,742,508,98,25,57.750854,07-feb-2023,840,533,1566,1134,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,100,0,0.000000,0
143,1669,449,243,49,96.855301,01/10/2022,1912,498,3507,2480,...,379.309900,379.309900,0.0,0.0,0.000000,0.000000,107,1,379.309900,0


In [252]:
# for index in resultDF.index:
#     name = str(resultDF["imageName"][index]) + ".jpeg"
#     resultDF["imageName"][index]=name
# resultDF

/tmp/ipykernel_12350/4228233195.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultDF["imageName"][index]=name


,left,top,width,height,conf,text,x,y,PageHeight,PageWidth,...,invoice date,invoice dt,due date,payment due date,ack date,ack dt,imageName,contextPresent,contextDist,output
72,1713,256,203,28,96.087402,21-07-2022,1916,284,3504,2479,...,688.104643,688.104643,0.0,0.0,0.000000,0.000000,1.jpeg,1,688.104643,0
73,1713,256,203,28,96.087402,21-07-2022,1916,284,3504,2479,...,688.104643,688.104643,0.0,0.0,0.000000,0.000000,1.jpeg,1,688.104643,0
206,1648,1936,214,31,79.921706,1-oct-22. to 31,1862,1967,3504,2479,...,0.000000,1077.894707,0.0,0.0,2156.320013,2156.320013,2.jpeg,1,1077.894707,0
204,1648,1936,136,31,48.239994,1-oct-22.,1784,1967,3504,2479,...,0.000000,1049.665185,0.0,0.0,2099.243673,2099.243673,2.jpeg,1,1049.665185,0
203,376,486,163,35,95.528168,22-sep-22,539,521,3504,2479,...,0.000000,0.000000,0.0,0.0,273.029302,273.029302,2.jpeg,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2133,1209,185,30,95.900703,01-feb-2023,2318,1239,3300,2550,...,0.000000,1553.208936,0.0,0.0,0.000000,0.000000,97.jpeg,1,1553.208936,0
270,2098,536,159,28,96.263855,21-08-2022,2257,564,3508,2479,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,98.jpeg,0,0.000000,0
88,742,508,98,25,57.750854,07-feb-2023,840,533,1566,1134,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,100.jpeg,0,0.000000,0
143,1669,449,243,49,96.855301,01/10/2022,1912,498,3507,2480,...,379.309900,379.309900,0.0,0.0,0.000000,0.000000,107.jpeg,1,379.309900,0


In [256]:
# os.chdir('/home/aman/Documents/Tally/Git-Document-AI/Document-AI/InvoiceDateModel-v2/Dataset/')

In [257]:
# pwd

'/home/aman/Documents/Tally/Git-Document-AI/Document-AI/InvoiceDateModel-v2/Dataset'

In [258]:
# resultDF.to_csv("tesseract2.csv")